In [72]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os
import glob

In [73]:
all_files = glob.glob(os.path.join('csv/', "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files))
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace('-', '_')
df.drop('unnamed: 64', axis=1, inplace=True)

# Turn dates into datetime.
date_cols = df.columns[df.columns.str.contains('date')]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

def original_sentence_format(x):
    try:
        year = int(x[:3])
        month = int(x[5:7]) / 12
        day = int(x[10:12]) / 365
        return year + month + day
    except:
        return 0

df['original_sentence'] = df.original_sentence.apply(original_sentence_format)
df['target'] = (df.original_sentence >= 1) * 1
df['age'] = df.case_date.apply(lambda x: x.year) - df.birthdate.apply(lambda x: x.year)
df.offense_type.replace(df.offense_type.value_counts()[14:].index, 'None', inplace=True)
df.reset_index(drop=True, inplace=True)
df = df[['target', 'age', 'offense_type', 'race', 'sex', 'addr_zip_code']]

/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_65406/4228300859.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))
/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_65406/4228300859.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))
/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_65406/4228300859.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))
/var/folders/7b/prbmv40d5wg4ymrn9dr3c76c0000gn/T/ipykernel_65406/4228300859.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files))


In [91]:
print(f'There are {df.addr_zip_code.nunique()} unique Zip Codes in the data')

There are 4750 unique Zip Codes in the data


In [90]:
df.addr_zip_code.value_counts()[:300]

78207    33321
0        30789
78237    17552
78228    16876
78210    15589
         ...  
77022       22
77002       22
75061       22
77060       22
78736       22
Name: addr_zip_code, Length: 300, dtype: int64

Use this website to extract zip code information: http://www.city-data.com/zips/78207.html

In [81]:
df.groupby('addr_zip_code').count()

,target,age,offense_type,race,sex
addr_zip_code,,,,,
0,30789,30313,30789,30789,30789
1,4,4,4,4,4
2,6,6,6,6,6
7,4,4,4,4,4
78,32,32,32,32,32
...,...,...,...,...,...
99654,2,2,2,2,2
99662,1,1,1,1,1
99664,2,2,2,2,2


In [75]:
pd.get_dummies(df,columns=['sex', 'race', 'offense_type', ], drop_first=True,)

,target,age,addr_zip_code,sex_F,sex_M,sex_U,race_A,race_B,race_C,race_H,...,offense_type_F2,offense_type_F3,offense_type_FC,offense_type_FS,offense_type_M,offense_type_MA,offense_type_MB,offense_type_MC,offense_type_None,offense_type_U
0,0,111.0,78209,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,111.0,78209,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,19.0,78218,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,28.0,78218,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1,21.0,78218,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399634,1,26.0,78240,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
399635,1,45.0,78201,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
399636,1,26.0,78227,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
399637,1,35.0,78240,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


1st goal predicting the time of the sentence (regression) - use subset of those who were sentenced.

2nd categorical variable if they were sentenced or not. 

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)